In [1]:

file_id = '1tTCOug6ktdMLOtKFtvRoCIs-U6ywXBaj'

In [2]:
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': file_id})
# Download the file to a local disc
download.GetContentFile('data.csv')

In [4]:
import pandas as pd
import numpy as np

In [5]:
# Read the CSV file without headers
df = pd.read_csv('data.csv', header=None)

# Assign headers 'class' and 'sentence'
df.columns = ['class', 'sentence']

# Now you can use the modified DataFrame 'df' for further processing


In [6]:
!pip install transformers torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.1 MB/s eta 0:00:00


In [40]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

In [8]:
# Specify the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df['class'].nunique())


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [17]:
df_full = df.copy()

In [32]:
df= df_full.sample(10000)
df['class'].nunique()
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['class'])

In [33]:
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)


In [34]:
max_length = 128
train_encodings = tokenizer(list(train_df['sentence']), truncation=True, padding=True, max_length=max_length)

In [35]:
val_encodings = tokenizer(list(val_df['sentence']), truncation=True, padding=True, max_length=max_length)


In [36]:
# Convert the tokenized data to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(train_df['label'].values))

In [37]:
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']),
                            torch.tensor(val_df['label'].values))

In [38]:
# Create data loaders for training and validation
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)


In [41]:
# Set the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=2e-5)
from sklearn.metrics import accuracy_score, f1_score

# Training loop
epochs = 3
model.to(device)
model.train()
for epoch in range(epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    all_predicted_labels = []
    all_true_labels = []

    for batch in train_dataloader:
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': batch[2].to(device)}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        predicted_labels = torch.argmax(outputs.logits, dim=1)
        true_labels = inputs['labels']
        total_loss += loss.item()
        total_correct += (predicted_labels == true_labels).sum().item()
        total_samples += len(true_labels)
        all_predicted_labels.extend(predicted_labels.tolist())
        all_true_labels.extend(true_labels.tolist())

    # Calculate accuracy and F1 score
    epoch_loss = total_loss / len(train_dataloader)
    epoch_accuracy = total_correct / total_samples
    epoch_f1_score = f1_score(all_true_labels, all_predicted_labels, average='weighted')

    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.4f} - F1 Score: {epoch_f1_score:.4f}")


Epoch 1/3 - Loss: 0.0687 - Accuracy: 0.9833 - F1 Score: 0.9833
Epoch 2/3 - Loss: 0.0455 - Accuracy: 0.9896 - F1 Score: 0.9896
Epoch 3/3 - Loss: 0.0363 - Accuracy: 0.9907 - F1 Score: 0.9907


In [42]:

# Evaluation
model.eval()
all_predicted_labels = []
all_true_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': batch[2].to(device)}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        all_predicted_labels.extend(predicted_labels.cpu().numpy())
        all_true_labels.extend(inputs['labels'].cpu().numpy())

# Calculate accuracy and F1 score
val_accuracy = accuracy_score(all_true_labels, all_predicted_labels)
val_f1_score = f1_score(all_true_labels, all_predicted_labels, average='weighted')

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation F1 Score: {val_f1_score:.4f}")


Validation Accuracy: 0.9563
Validation F1 Score: 0.9563


19464                     Books
25891                     Books
48700               Electronics
17877                 Household
35536    Clothing & Accessories
                  ...          
16476                 Household
3791                  Household
42652               Electronics
18423                 Household
8456                  Household
Name: class, Length: 10000, dtype: object